In [ ]:
!pip install torch pandas scikit-learn

In [66]:
import pandas as pd
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD, SMAIndicator, EMAIndicator, ADXIndicator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load your data
data = pd.read_csv(r'C:\Users\vigop\BinaryOptionsTools\history-AUDNZD_otc.csv')
# Calculate MACD
macd = MACD(close=data['close'])
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()

# Calculate Moving Averages (SMA and EMA)


# Drop any NaN values generated by the indicators
data.dropna(inplace=True)

# Define the feature columns, including the new signals
features = ['open', 'high', 'low', 'close','macd', 'macd_signal']

# Define a future prediction window (e.g., 5 periods ahead)
prediction_window = 5

# Create the target column: 1 if future close price is higher, else 0
data['target'] = (data['close'].shift(-prediction_window) > data['close']).astype(int)

# Extract features and target
X = data[features].values
y = data['target'].values

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=42)


In [67]:
import torch
import torch.nn as nn
import torch.optim as optim

class BinaryOptionsLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BinaryOptionsLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Only last time step's output is needed
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out


In [6]:
# Hyperparameters
input_size = len(features)  # Update input size based on the number of features
hidden_size = 64
num_layers = 2
output_size = 1
num_epochs = 100
batch_size = 64
learning_rate = 0.001

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# DataLoader for batch processing
train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Model, loss function, and optimizer
model = BinaryOptionsLSTM(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Save the trained model
torch.save(model.state_dict(), 'binary_options_lstm.pth')


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).cpu().numpy()
    y_pred_binary = (y_pred > 0.5).astype(int)

# Convert predictions and true labels to binary
y_test_binary = y_test_tensor.cpu().numpy()

# Calculate evaluation metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
precision = precision_score(y_test_binary, y_pred_binary)
recall = recall_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')


In [72]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a basic neural network
class BinaryOptionsModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BinaryOptionsModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

# Hyperparameters
input_size = 6
hidden_size = 64
output_size = 1
num_epochs = 50
batch_size = 512
learning_rate = 0.001

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# DataLoader for batch processing
train_data = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = BinaryOptionsModel(input_size, hidden_size, output_size)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50], Loss: 0.6890
Epoch [2/50], Loss: 0.6961
Epoch [3/50], Loss: 0.6935
Epoch [4/50], Loss: 0.6940
Epoch [5/50], Loss: 0.6907
Epoch [6/50], Loss: 0.6935
Epoch [7/50], Loss: 0.6904
Epoch [8/50], Loss: 0.6969
Epoch [9/50], Loss: 0.6918
Epoch [10/50], Loss: 0.6911
Epoch [11/50], Loss: 0.6927
Epoch [12/50], Loss: 0.6934
Epoch [13/50], Loss: 0.6962
Epoch [14/50], Loss: 0.6946
Epoch [15/50], Loss: 0.6872
Epoch [16/50], Loss: 0.6895
Epoch [17/50], Loss: 0.6896
Epoch [18/50], Loss: 0.6874
Epoch [19/50], Loss: 0.6909
Epoch [20/50], Loss: 0.6967
Epoch [21/50], Loss: 0.6871
Epoch [22/50], Loss: 0.6935
Epoch [23/50], Loss: 0.6940
Epoch [24/50], Loss: 0.6885
Epoch [25/50], Loss: 0.6916
Epoch [26/50], Loss: 0.6892
Epoch [27/50], Loss: 0.6901
Epoch [28/50], Loss: 0.6929
Epoch [29/50], Loss: 0.6910
Epoch [30/50], Loss: 0.6943
Epoch [31/50], Loss: 0.6928
Epoch [32/50], Loss: 0.6930
Epoch [33/50], Loss: 0.6932
Epoch [34/50], Loss: 0.6855
Epoch [35/50], Loss: 0.6891
Epoch [36/50], Loss: 0.6889
E

In [73]:
# Evaluate the model on test data
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(X_test_tensor)
    predicted = (outputs > 0.5).float()
    print(predicted)
    accuracy = (predicted.eq(y_test_tensor).sum().item()) / y_test_tensor.size(0)
    print(f'Accuracy: {accuracy * 100:.2f}%')


tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])
Accuracy: 51.76%


In [59]:
# Define the path where you want to save the model
model_path = "binary_options_model2.pth"

# Save the model's state_dict (parameters)
torch.save(model.state_dict(), model_path)

print(f"Model saved to {model_path}")


Model saved to binary_options_model2.pth


In [45]:
# Initialize the model architecture (same as before)
loaded_model = BinaryOptionsModel(input_size, hidden_size, output_size)

# Load the saved model parameters
loaded_model.load_state_dict(torch.load("trend.pth"))

# Set the model to training mode (required if you want to train)
loaded_model.train()

print("Model loaded for retraining")


Model loaded for retraining


C:\Users\vigop\AppData\Local\Temp\ipykernel_18244\3150394980.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("trend.pth"))


In [46]:
# Set up the optimizer
optimizer = optim.Adam(loaded_model.parameters(), lr=learning_rate)

# Optionally, if you had learning rate schedules, reinitialize them as well.


In [47]:
# Training loop (same as before)
for epoch in range(100):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward pass
        outputs = loaded_model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/50], Loss: 0.3536
Epoch [2/50], Loss: 0.2920
Epoch [3/50], Loss: 0.3057
Epoch [4/50], Loss: 0.3397
Epoch [5/50], Loss: 0.4785
Epoch [6/50], Loss: 0.3532
Epoch [7/50], Loss: 0.3126
Epoch [8/50], Loss: 0.3380
Epoch [9/50], Loss: 0.2750
Epoch [10/50], Loss: 0.4019
Epoch [11/50], Loss: 0.2210
Epoch [12/50], Loss: 0.2037
Epoch [13/50], Loss: 0.3933
Epoch [14/50], Loss: 0.2925
Epoch [15/50], Loss: 0.2695
Epoch [16/50], Loss: 0.3443
Epoch [17/50], Loss: 0.3477
Epoch [18/50], Loss: 0.2602
Epoch [19/50], Loss: 0.2419
Epoch [20/50], Loss: 0.3082
Epoch [21/50], Loss: 0.3136
Epoch [22/50], Loss: 0.3185
Epoch [23/50], Loss: 0.3659
Epoch [24/50], Loss: 0.3791
Epoch [25/50], Loss: 0.2381
Epoch [26/50], Loss: 0.3016
Epoch [27/50], Loss: 0.3576
Epoch [28/50], Loss: 0.2577
Epoch [29/50], Loss: 0.3079
Epoch [30/50], Loss: 0.3470
Epoch [31/50], Loss: 0.3742
Epoch [32/50], Loss: 0.2899
Epoch [33/50], Loss: 0.3656
Epoch [34/50], Loss: 0.3813
Epoch [35/50], Loss: 0.2937
Epoch [36/50], Loss: 0.3950
E

In [41]:
# Save the retrained model
torch.save(loaded_model.state_dict(), "binary_options_model_retrained2.pth")
print("Retrained model saved")


Retrained model saved


In [48]:
# Evaluate the model on test data
loaded_model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = loaded_model(X_test_tensor)
    predicted = (outputs > 0.5).float()
    print(predicted)
    accuracy = (predicted.eq(y_test_tensor).sum().item()) / y_test_tensor.size(0)
    print(f'Accuracy: {accuracy * 100:.2f}%')


tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.]])
Accur

In [71]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

4673